In [1]:
def formatTime(t):
  if t < 60:
    return "{:.2f} sec".format(t)
  elif t < (60*60):
    return "{:.2f} min".format(t/60 )
  elif t < (60*60*24):
    return  "{:.2f} hour".format((t/60)/24)

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 62kB/s 
     |████████████████████████████████| 204kB 43.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1d57e180c461c87239a59c0c4925942c708a2a0312f0345fea6d12d192093ccc
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
def formatTime(t):
  if t < 60:
    return "{:.2f} sec".format(t)
  elif t < (60*60):
    return "{:.2f} min".format(t/60 )
  elif t < (60*60*24):
    return  "{:.2f} hour".format((t/60)/24)

In [4]:
import sys
import random
import time
import binascii
import math
from pyspark import SparkConf, SparkContext
import collections
import numpy as np
from itertools import islice, count
from sympy import isprime, nextprime

In [5]:
conf = SparkConf().setMaster("local").setAppName("Local Sensitive Hashing")
sc = SparkContext.getOrCreate(conf=conf)

In [6]:
begin = time.time()
#read data
rdd = sc.textFile("article_text.txt")
doc = rdd.collect()
# input

doc_input = rdd.collect()[0]
doc.insert(0,doc_input)
rdd = sc.parallelize(doc)

In [7]:
doc_rdd = rdd.map(lambda x: x.lower().replace('.','').replace(',','')).filter(lambda x: x!=" ").filter(lambda x: x!='\n')
doc = doc_rdd.collect()
print(doc)
print("Length of doc: ", len(doc))

['nội dung bài viết dự thảo quy chế xét tuyển đại học cao đẳng 2021: thuận lợi hơn cho thí sinh: theo đó dự thảo quy định để xét tuyển đợt 1 thí sinh đăng ký nguyện vọng xét tuyển vào đại học cao đẳng ngành giáo dục mầm non chỉ được chọn một trong hai phương thức: đăng ký trực tiếp trên phiếu và đăng ký trực tuyến trong các đợt xét tuyển sau đợt 1 thí sinh đăng ký và nộp lệ phí theo quy định của trườngsau khi có kết quả thi tốt nghiệp thpt thí sinh được điều chỉnh nguyện vọng đăng ký xét tuyển ba lần trong thời gian quy định bằng phương thức trực tuyến\xa0thí sinh xác nhận nhập học vào trường đã trúng tuyển trong thời hạn quy định bằng cách chỉ gửi bản chính giấy chứng nhận kết quả thi (đối với thí sinh tham dự kỳ thi thpt của năm tuyển sinh) đến trường trực tiếp hoặc bằng hình thức thư chuyển phát nhanh thí sinh đã xác nhận nhập học thì không được tham gia xét tuyển ở\xa0trường khácmột nội dung sửa đổi đáng chú ý của dự thảo quy chế tuyển sinh năm nay là hội đồng tuyển sinh của trường

In [8]:
shinglingBeginTime = time.time()

In [9]:
ngram = 9
#Shingling
def shingling(document):
    shingle = set()
    for j in range(len(document)-(ngram-1)):
        # Encode shingle to crc32
        crc = binascii.crc32(document[j:(ngram+j)].encode())
        shingle.add(crc)
    return shingle

In [10]:
# Shingle doc 
shingleDocSetRDD = doc_rdd.map(lambda x : shingling(x))
# get number of shingle
sumOfShingle = sum(shingleDocSetRDD.map(lambda x: len(x)).collect())

shingleDocSet = shingleDocSetRDD.collect()

In [11]:
print("Sum of shingle is ", sumOfShingle)

Sum of shingle is  97703


In [12]:
shinglingEndTime = time.time()

In [13]:
print("Shingling end in {}".format(formatTime(shinglingEndTime - shinglingBeginTime)))

Shingling end in 9.42 sec


In [14]:
hashingBeginTime = time.time()

In [15]:
k = 500

In [16]:
# Get prime number for MinHashing

nextPrime = nextprime(sumOfShingle)

# Generate k random integer to hash shingle
def randomHashFunc(k):
    return random.sample(range(sumOfShingle), k)

# Initialize A and B with k random integer
randomAHashFunc = randomHashFunc(k)
randomBHashFunc = randomHashFunc(k)

In [17]:
print(nextPrime)

97711


In [18]:
def minHashShingle(shingleDoc, a, b):
  minHash = np.Inf
  for x in shingleDoc:
        # hashCode implementation based on Standford slide 
        hashCode = ((a * x + b) % nextPrime) % sumOfShingle
        # if hashCode < minHash then hashCode is minHash of current shingle
        minHash = hashCode if hashCode < minHash else minHash
  return minHash

# Make signature
def signatureGenerator(shingleDoc):
  signature = []
  
  # Make k signature length
  for i in range(0, k):
    minHash = minHashShingle(shingleDoc, randomAHashFunc[i], randomBHashFunc[i])
    signature.append(minHash)
  return signature

In [19]:
signatures = shingleDocSetRDD.map(lambda x: signatureGenerator(x)).collect()
print(signatures)

[[39, 26, 3, 63, 209, 28, 32, 68, 53, 45, 238, 0, 2, 48, 80, 26, 53, 1, 15, 51, 49, 5, 27, 225, 26, 3, 42, 59, 18, 14, 19, 43, 59, 12, 16, 84, 30, 6, 55, 93, 14, 4, 109, 35, 70, 2, 37, 57, 37, 25, 39, 1, 81, 4, 22, 68, 74, 19, 0, 74, 59, 105, 62, 7, 150, 55, 0, 134, 80, 22, 4, 193, 4, 26, 69, 6, 36, 75, 19, 30, 51, 237, 239, 50, 51, 26, 22, 9, 72, 24, 355, 25, 0, 23, 13, 126, 134, 8, 302, 16, 80, 1, 62, 11, 21, 19, 4, 9, 16, 71, 296, 11, 70, 44, 2, 81, 6, 93, 124, 21, 41, 7, 176, 5, 3, 70, 86, 15, 139, 273, 59, 78, 31, 42, 55, 2, 53, 22, 137, 1, 124, 0, 5, 0, 74, 44, 5, 23, 82, 86, 134, 136, 37, 0, 2, 0, 41, 84, 67, 55, 42, 96, 1, 55, 45, 35, 99, 48, 1, 2, 142, 51, 122, 94, 1, 113, 25, 271, 35, 36, 26, 10, 4, 25, 11, 136, 4, 5, 101, 115, 22, 301, 1, 98, 53, 144, 1, 3, 11, 27, 182, 95, 62, 148, 57, 4, 16, 9, 0, 48, 8, 20, 33, 6, 240, 29, 27, 125, 34, 68, 63, 1, 174, 10, 61, 54, 65, 31, 16, 185, 33, 24, 144, 20, 152, 11, 201, 82, 24, 3, 47, 93, 19, 111, 7, 45, 1, 56, 134, 45, 110, 52, 5,

In [20]:
signaturesMatrix = np.array(signatures)
print(signaturesMatrix)

[[ 39  26   3 ...  59  15  42]
 [ 39  26   3 ...  59  15  42]
 [ 79 164  21 ...   1   2   6]
 ...
 [ 70   2   3 ...  11  15  49]
 [  4   1   3 ...  18   7   4]
 [ 69 232  56 ...  18  43   6]]


In [21]:
hashingEndTime = time.time()

In [22]:
print("MinHashing end in {}".format(formatTime(hashingEndTime - hashingBeginTime)))

MinHashing end in 32.45 sec


In [23]:
LSHBeginTime = time.time()

In [24]:
b = 10
r = 10
# s = (1/b)**(1/r)
s = 0.8
bucket = []

In [25]:
def jaccardSim(A,B):
    return sum(A==B) / len(A)

def divideBands(x, signaturesMatrix):
  return signaturesMatrix.T[x:x+r]

In [26]:
# Devide bands
rdd3 = sc.parallelize(list(range(0,k,r)))
bands = rdd3.map(lambda x: divideBands(x, signaturesMatrix))

In [27]:
bands.count()

50

In [28]:
# Generate Pair
pairs = [(0,i) for i in range(1,len(shingleDocSet))]

In [29]:
# Insert pair to bukect if sim(pair) reach threshold s
def getBuckets(band):
  res = []
  for pair in pairs:
      js = jaccardSim(band.T[pair[0]], band.T[pair[1]])
      if js > s:
          res.append((pair, 1))
  return res

In [30]:
buckets = sc.parallelize(bands.map(lambda x: getBuckets(x)).reduce(lambda x,y: x+y))
print(buckets.collect())

[((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1), ((0, 1), 1)]


In [31]:
findCandidatePairs = buckets.reduceByKey(lambda x,y: x+y)
findCandidatePairs.sortBy(lambda x: x[1], False)
candidatePairs = findCandidatePairs.collect()
print(candidatePairs)

[((0, 1), 50)]


In [32]:
LSHEndTime = time.time()

In [33]:
print("LSH end in {}".format(formatTime(LSHEndTime - LSHBeginTime)))

LSH end in 20.93 sec


In [34]:
print("candidate pairs: ", candidatePairs)

candidate pairs:  [((0, 1), 50)]


In [35]:
doc_input

'Nội dung bài viết Dự thảo quy chế xét tuyển đại học, cao đẳng 2021: Thuận lợi hơn cho thí sinh: Theo đó, dự thảo quy định để xét tuyển đợt 1, thí sinh đăng ký nguyện vọng xét tuyển vào đại học, cao đẳng ngành giáo dục mầm non chỉ được chọn một trong hai phương thức: đăng ký trực tiếp trên phiếu và đăng ký trực tuyến. Trong các đợt xét tuyển sau đợt 1, thí sinh đăng ký và nộp lệ phí theo quy định của trường.Sau khi có kết quả thi tốt nghiệp THPT, thí sinh được điều chỉnh nguyện vọng đăng ký xét tuyển ba lần trong thời gian quy định bằng phương thức trực tuyến.\xa0Thí sinh xác nhận nhập học vào trường đã trúng tuyển trong thời hạn quy định bằng cách chỉ gửi bản chính giấy chứng nhận kết quả thi (đối với thí sinh tham dự kỳ thi THPT của năm tuyển sinh) đến trường trực tiếp hoặc bằng hình thức thư chuyển phát nhanh. Thí sinh đã xác nhận nhập học thì không được tham gia xét tuyển ở\xa0trường khác.Một nội dung sửa đổi đáng chú ý của dự thảo quy chế tuyển sinh năm nay là hội đồng tuyển sinh 

In [36]:
for i in candidatePairs:
  print("doc[0] is similarity to doc[{}]\n".format(i[0][1]))
  print(doc[i[0][1]])

doc[0] is similarity to doc[1]

nội dung bài viết dự thảo quy chế xét tuyển đại học cao đẳng 2021: thuận lợi hơn cho thí sinh: theo đó dự thảo quy định để xét tuyển đợt 1 thí sinh đăng ký nguyện vọng xét tuyển vào đại học cao đẳng ngành giáo dục mầm non chỉ được chọn một trong hai phương thức: đăng ký trực tiếp trên phiếu và đăng ký trực tuyến trong các đợt xét tuyển sau đợt 1 thí sinh đăng ký và nộp lệ phí theo quy định của trườngsau khi có kết quả thi tốt nghiệp thpt thí sinh được điều chỉnh nguyện vọng đăng ký xét tuyển ba lần trong thời gian quy định bằng phương thức trực tuyến thí sinh xác nhận nhập học vào trường đã trúng tuyển trong thời hạn quy định bằng cách chỉ gửi bản chính giấy chứng nhận kết quả thi (đối với thí sinh tham dự kỳ thi thpt của năm tuyển sinh) đến trường trực tiếp hoặc bằng hình thức thư chuyển phát nhanh thí sinh đã xác nhận nhập học thì không được tham gia xét tuyển ở trường khácmột nội dung sửa đổi đáng chú ý của dự thảo quy chế tuyển sinh năm nay là hội đồ

In [40]:
end = time.time()

In [ ]:
print("Time running: {} sec".format(formatTime(end-begin)))